In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header

import time

# = = = = = = = = = = = = = = =

import math

In [2]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3fd3LhLaJKGh1SH6VEp'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/input.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：2



,No,Url
0,1,https://web.tecalliance.net/tecdoc/qa/parts/se...
1,2,https://web.tecalliance.net/tecdoc/qa/parts/se...


In [4]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                     1
 Url    https://web.tecalliance.net/tecdoc/qa/parts/se...
 Name: 0, dtype: object)

In [5]:
url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)

data = {'getArticles': {'applyDqmRules': True,
                        'articleCountry': 'DE',
                        'articleStatusIds': [],
                        'criteriaFilters': [],
                        'dataSupplierIds': [int(dict_param['brands'][0].strip())],
                        'filterQueries': ['(dataSupplierId NOT IN (4978,4982))'],
                        'genericArticleIds': [int(dict_param['groups'][0].strip())],
                        'includeAccessoryArticles': False,
                        'includeAll': False,
                        'includeArticleCriteria': True,
                        'includeArticleLogisticsCriteria': True,
                        'includeArticleStatusFacets': True,
                        'includeArticleText': True,
                        'includeComparableNumbers': True,
                        'includeCriteriaFacets': True,
                        'includeDataSupplierFacets': True,
                        'includeGTINs': True,
                        'includeGenericArticleFacets': True,
                        'includeGenericArticles': True,
                        'includeImages': True,
                        'includeLinkages': True,
                        'includeLinks': True,
                        'includeMisc': True,
                        'includeOEMNumbers': True,
                        'includePDFs': True,
                        'includePartsListArticles': False,
                        'includePrices': True,
                        'includeReplacedByArticles': True,
                        'includeReplacesArticles': True,
                        'includeTradeNumbers': True,
                        'lang': 'qa',
                        'linkagesPerPage': 100,
                        'page': 1,
                        'perPage': 1_000,
                        'provider': 23365,
                        'sort': [{'field': 'articleNumber', 'direction': 'asc'}]}}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = f'https://web.tecalliance.net/tecdoc/qa/parts/search?brands={dict_param['brands'][0].strip()}&groups={dict_param['groups'][0].strip()}'

        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(**{'x-api-key': api}),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                print(f'fuck api {api}')
                time.sleep(3)
                with open('./api.txt', 'r') as api_txt:
                    api = api_txt.read()
            elif dict_['status'] == 200:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

resp

1


<Response [200]>

In [6]:
count = dict_['totalMatchingArticles']

count

658

In [7]:
count <= 0

False

In [8]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Url': url_check,
                          'SKU Count': count,
                          'Page': math.ceil(count / 1_000)})

list_dict_correct

[{'No': 1,
  'Url': 'https://web.tecalliance.net/tecdoc/qa/parts/search?brands=21&groups=298',
  'Page': 1}]

In [9]:
crawler_status = True

crawler_status

True

In [10]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                           1
 Url          https://web.tecalliance.net/tecdoc/qa/parts/se...
 Check_Url    https://web.tecalliance.net/tecdoc/qa/parts/se...
 Name: 0, dtype: object]

In [11]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_page.xlsx', index=False)

df_correct

,No,Url,Page
0,1,https://web.tecalliance.net/tecdoc/qa/parts/se...,1


In [12]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_page_error.xlsx', index=False)

df_error

,No,Url,Check_Url
0,1,https://web.tecalliance.net/tecdoc/qa/parts/se...,https://web.tecalliance.net/tecdoc/qa/parts/se...
